# Image Filters and Feature Enhancement
*Author: Vladislav Kim*
* [Introduction](#intro)
* [Edge enhancement](#edge)
* [Blob detection](#spot)


<a id="intro"></a> 
## Introduction
In addition to denoising, thresholding and background subtraction that were covered in the [previous](https://github.com/vladchimescu/bioimg/blob/master/Jupyter/2-image-transformation.ipynb) notebook, there are a number of feature enhancing image transformations that can be useful for microscopy image analysis. In many applications we are interested in finding edges or enhancing object boundaries. Also common in microscopy are spherical shapes, e.g. nuclei, which can be detected using Laplace-of-Gaussian (LoG) operator.

In [ ]:
# load third-party Python modules
import javabridge
import bioformats as bf
import skimage
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn

import sys
sys.path.append('..')

javabridge.start_vm(class_path=bf.JARS)

We will start by loading an image stack of B and T cells with stained nuclei (Hoechst), viability marker (Calcein) and surface markers (APC, PE).

In [ ]:
from base.utils import load_imgstack
imgstack = load_imgstack(fname="data/BiTE/Tag2-r04c02f1.tiff")

In [ ]:
mip=np.max(imgstack, axis=0)

In [ ]:
from base.plot import plot_channels
channels = ['PE', 'Calcein', 'Hoechst', 'APC']
plot_channels([mip[:,:,i]**0.5 for i in range(4)],
              nrow=1, ncol=4, titles=channels)

<a id="edge"></a> 
## Edge enhancement
We will apply edge enhancement on the APC channel with the stained B cell surface marker:

In [ ]:
apc = mip[:,:,3]**0.3

Edges are object boundaries such as the interface between a foreground object (e.g. a cell) and the background, which can be detected using gradient operators. In `skimage.filters` module there is a number of image gradient approximations such as Sobel, Roberts, Scharr, and Prewitt operators

In [ ]:
from skimage.filters import sobel

In [ ]:
plot_channels([apc, sobel(apc)],
             nrow=1, ncol=2,
             titles=['Original', 'Sobel edge enhancement'],
             cmap='gray', scale_x=7, scale_y=7)

After Sobel operator is applied, the image has more pronounced edges, i.e. cell surface boundaries appear enhanced. Note that the resulting image is not binary and has a distribution shifted towards zero as non-edge pixels are suppresed:

In [ ]:
plt.figure(figsize=(6,6))
sn.distplot(apc, kde=False, label='Original image')
sn.distplot(sobel(apc), kde=False, label='Sobel edge enhancement')
plt.xlabel('Intensity')
plt.legend()

We can achieve a similar result by computing the magnitude of the image gradient:
$$ |\nabla g| = \sqrt{\left(\frac{\partial g}{\partial x}\right)^2 + \left(\frac{\partial g}{\partial y}\right)^2} $$

In [ ]:
# compute x and y components of the gradient
g_x, g_y = np.gradient(apc)
# magnitude of the gradient
g_norm = np.sqrt(g_x**2 + g_y**2)

In [ ]:
plt.figure(figsize=(8,8))
plt.imshow(g_norm, cmap='gray')
plt.title('Gradient image')
plt.axis('off')

<a id="spot"></a>
## Blob and circle detection
There is a number of methods for blob or spot detection, most of which are based on Laplace-of-Gaussian of the image pyramid. First we will adjust the brightness of the image of the nuclei using gamma correction.

In [ ]:
hoechst = mip[:,:,2]**0.4

In [ ]:
from transform.process import threshold_img
from skimage.feature import blob_log

We will apply blob detection in a thresholded image to suppress noise:

In [ ]:
img_th = threshold_img(hoechst, method='otsu')
blobs = blob_log(img_th,
                 min_sigma=10, max_sigma=12, threshold=0.05)

In [ ]:
# number of detected blobs
blobs.shape

In each row of `blobs` we have $(x,y)$-coordinates of the blob center and its radius. We can plot the detected spots over the original image of the nuclei. The blobs will be visualized as teal circles.

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
for blob in blobs:
    y, x, r = blob
    c = plt.Circle((x, y), r, color='cyan', linewidth=1.2, fill=False)
    ax.add_patch(c)
ax.imshow(hoechst, cmap='gray')
ax.axis('off')

In addition to Laplace-of-Gaussian blob detection we can use Hough transform to detect circles:


In [ ]:
from skimage.transform import hough_circle, hough_circle_peaks
from skimage.feature import canny

In [ ]:
# radius grid
rgrid = np.arange(10,12)
hspace = hough_circle(threshold_img(sobel(hoechst), method='otsu', binary=True), rgrid)
accums, cx, cy, radii = hough_circle_peaks(hspace, rgrid,num_peaks=1000)

In [ ]:
'''plt.figure(figsize=(8,8))
plt.imshow(threshold_img(sobel(hoechst), method='otsu', binary=True))'''

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
for y, x, r in zip(cy[:1000], cx[:1000], radii[:1000]):
    c = plt.Circle((x, y), r, color='cyan', linewidth=1.2, fill=False)
    ax.add_patch(c)
ax.imshow(hoechst, cmap='gray')

## Local Intensity Maxima
Sometimes it is useful to find local intensity peaks for example so that we can provide these intensity maxima as seeds for a segmentation algorithm

Here we can talk more about white and black tophat filters